# Preprocessing PDF


In [14]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [15]:

local_path = "PDF/JPTK-1.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [16]:
# Preview first page
text = data[0].page_content
print(text)

ISSN: 2303-288X

Vol. 2, No. 1, April 2013

PENGARUH MODEL PEMBELAJARAN KOOPERATIF TIPE JIGSAW DAN MOTIVASI BELAJAR TERHADAP PRESTASI BELAJAR ILMU FISIOLOGI OLAHRAGA

Made Budiawan¹, Ni Luh Kadek Alit Arsani² ¹·² Jurusan Ilmu Keolahragaan Fakultas Olahraga dan Kesehatan Universitas Pendidikan Ganesha Singaraja, Indonesia e-mail: budiawan_ajus@yahoo.co.id, alit_arsani@yahoo.com

Abstrak

Tujuan penelitian ini adalah untuk mengetahui: 1) Pengaruh model pembelajaran Kooperatif tipe Jigsaw terhadap prestasi belajar Ilmu Fisiologi Olahraga. 2) Pengaruh motivasi terhadap prestasi belajar Ilmu Fisiologi Olahraga. 3) Interaksi antara model pembelajaran Kooperatif tipe Jigsaw dan motivasi terhadap prestasi belajar Ilmu Fisiologi Olahraga.

Metode penelitian yang digunakan adalah metode eksperimen. Penelitian dilaksanakan bulan April sampai Desember 2012. Populasi adalah mahasiswa Jurusan Ilmu Keolahragaan dan Pendidikan Kepelatihan Olahraga Fakultas Olahraga dan Kesehatan Universitas Pendidikan

# Vektorisasi


In [17]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [18]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

print(chunks)

[Document(metadata={'source': 'PDF/JPTK-1.pdf'}, page_content='ISSN: 2303-288X\n\nVol. 2, No. 1, April 2013\n\nPENGARUH MODEL PEMBELAJARAN KOOPERATIF TIPE JIGSAW DAN MOTIVASI BELAJAR TERHADAP PRESTASI BELAJAR ILMU FISIOLOGI OLAHRAGA\n\nMade Budiawan¹, Ni Luh Kadek Alit Arsani² ¹·² Jurusan Ilmu Keolahragaan Fakultas Olahraga dan Kesehatan Universitas Pendidikan Ganesha Singaraja, Indonesia e-mail: budiawan_ajus@yahoo.co.id, alit_arsani@yahoo.com\n\nAbstrak\n\nTujuan penelitian ini adalah untuk mengetahui: 1) Pengaruh model pembelajaran Kooperatif tipe Jigsaw terhadap prestasi belajar Ilmu Fisiologi Olahraga. 2) Pengaruh motivasi terhadap prestasi belajar Ilmu Fisiologi Olahraga. 3) Interaksi antara model pembelajaran Kooperatif tipe Jigsaw dan motivasi terhadap prestasi belajar Ilmu Fisiologi Olahraga.\n\nMetode penelitian yang digunakan adalah metode eksperimen. Penelitian dilaksanakan bulan April sampai Desember 2012. Populasi adalah mahasiswa Jurusan Ilmu Keolahragaan dan Pendidikan 

In [19]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings:  50%|█████     | 2/4 [00:44<00:45, 22.80s/it]

# Retrival


In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "gemma2"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""(Gunakan bahasa indonesia) Anda adalah model AI untuk menghasilkan teks berdasarkan database vektor. Berikan jawaban hanya berdasarkan database dokumen yang saya berikan di database vektor. Jika kamu tidak menemukan jawaban, berikan jawaban bahwa kamu tidak mempelajarinya, Buatkan jawaban dengan versi yang berbeda untuk setiap pertanyaan yang mirip atau bahkan sama. Selain itu untuk pertanyaan yang pertama gunakan salam pembuka "Salam Harmoni".
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = 'Apa tujuan penelitian tersebut'
chain.invoke(question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it]


'Berdasarkan teks yang kamu berikan, tujuan penelitian tersebut adalah untuk **mengetahui faktor-faktor yang memengaruhi kemampuan menulis mahasiswa pada mata kuliah Writing I dan mencari solusi untuk meningkatkan kemampuan menulis mereka.** \n\nPenelitian ini difokuskan pada:\n\n* Mengidentifikasi kesulitan mahasiswa dalam menulis, seperti masalah dalam merumuskan kalimat topik, mengorganisasikan ide, serta kesalahan ejaan dan tanda baca.\n* Memahami pengaruh metode pengajaran yang diterapkan (misalnya, portofolio dan pendekatan proses) terhadap kemampuan menulis mahasiswa. \n* Mencari solusi untuk mengatasi permasalahan tersebut dan meningkatkan kemampuan menulis mahasiswa di mata kuliah Writing I.\n\n\nSemoga membantu! \n'

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL halaman web yang ingin di-scrape
url = "https://ejournal.undiksha.ac.id/index.php/JPI/article/view/1414/1275"

# Mengirimkan permintaan HTTP ke URL
response = requests.get(url)

# Memeriksa apakah permintaan berhasil
if response.status_code == 200:
    # Parsing konten HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Menyimpan kode HTML ke dalam variabel
    html_content = soup.prettify()

    # Menyimpan HTML ke file .txt
    with open("output.txt", "w", encoding="utf-8") as file:
        file.write(html_content)
    print("HTML berhasil disimpan ke file 'output.txt'")
else:
    print(f"Permintaan gagal dengan status code {response.status_code}")


ConnectionError: HTTPSConnectionPool(host='ejournal.undiksha.ac.id', port=443): Max retries exceeded with url: /index.php/JPI/article/view/1414/1275 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000201AADCA850>: Failed to resolve 'ejournal.undiksha.ac.id' ([Errno 11001] getaddrinfo failed)"))